# Klasyfikacja za pomocą algorytmu wektorów wspierających (SVM)
## Materiały
Na tych ćwiczeniach zapoznamy się z zastosowaniem SVM do klasyfikacji.
Poniżej znajduje się fragment kodu dostarczający kilku funkcji,  z których dziś będziemy korzystać. 

In [ ]:
import numpy as np
import matplotlib.pylab as plt

#### Definicje funkcji pomocniczych, których nie musisz modyfikować

In [ ]:
def rysujDaneGrup(X, y, marker, xlabel, ylabel,legend_list):
    '''X - macierz wartości wejściowych zorganizowana tak, że kolejne przykłady są
    w wierszach, kolumny to kolejne wynmiary  wejścia,
    y - wektor określający przynależność do grupy, indeksy tego wektora odpowiadają wireszom macierzy X,
    marker - zestaw markerów do oznaczania elementów grup, markerów powinno być tyle ile jest grup'''
    p=[]
    for i,g in enumerate(np.unique(y)):
        g = int(g)
        tmp =plt.plot(X[np.where(y==g),0],X[np.where(y==g),1],marker[i])
        p.append(tmp[0])
    plt.legend(p,legend_list)
    # Dodajemy napisy
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    
def rysujPodzial(model, X):
    # wytworzymy siatkę punktów pokrywających obszar danych:
    N = 100 # ilość punktów siatki w jednym wymiarze
    os_x = np.linspace(X.min(),X.max(),N)
    klasa = np.zeros((N,N))
    for ix1, x1 in enumerate(os_x):
        for ix2, x2 in enumerate(os_x):
            XX = np.array([x1,x2]).reshape(1,2)
            klasa[ix1,ix2] = svmPredict(model, XX) # dla każdego punktu siatki obliczamy jego klasę
    
    x1_grid,x2_grid = np.meshgrid(os_x,os_x)
    plt.contourf(x1_grid, x2_grid, klasa.T,2)   
            
    
def gaussianKernel(Xi,Xj,sigma=0.1):
    z = np.dot((Xi-Xj).T,(Xi-Xj))
    S = 2*sigma*sigma
    Z= z/S
    return np.exp(-Z)
    

#### Funkcja do trenowania modelu SVM. 

**Uwaga**: To jest wersja uproszczona algorytmu wzorowana na
           https://www.microsoft.com/en-us/research/wp-content/uploads/2016/02/tr-98-14.pdf
           
w szczególności odwołuje się ona do numerów równań z tego artykułu.
           
Nie jest ani specjalnie elegancka ani szybka.

Do praktycznych zastosowań zaleca się stosowanie zoptymalizowanych bibliotek:
* LIBSVM   (http://www.csie.ntu.edu.tw/~cjlin/libsvm/)
* SVMLight (http://svmlight.joachims.org/)
* lub [sklearn](http://scikit-learn.org/stable/modules/svm.html)
    

In [ ]:
def svmTrain(X, Y, C, kernelFunction, tol = 1e-3, max_passes = 5, sigma=0.1):
    '''VMTRAIN Trenuje klasyfikator  SVM za pomocą uproszczonego algorytmu SMO.
         X - macierz wejściowa  przykładów z ciągu uczącego wiersze - przyklady, kolumny - cechy
         Y - wektor etykiet klas {-1,1}
         C  - regularyzacja SVM
         tol - tolerancja na odstępstwa od wrunków KTT
         max_passes - ile iteracji bez zmian mnożników Lagrangaea wykonać zanim uznamy, że już nie ma co poprawiać
        kernelFunction - funkcja jądra, zaimplementowane są:
            - gaussianKernel
            - linearKernel
        sigma - standardowe odchylenie dla jądra gaussowskiego
    
    funkcja zwraca parametry doapsowanego modelu w słowniku model
    '''

    # Pobieramy rozmiary
    m,n = X.shape #m - ilość przykładów, n - wymiar wejścia
  

    # Zmienne
    alphas = np.zeros(m)
    b = 0
    E = np.zeros(m)
    passes = 0
    eta = 0
    L = 0
    H = 0

    # Pre-compute the Kernel Matrix since our dataset is small
    # (in practice, optimized SVM packages that handle large datasets
    #  gracefully will _not_ do this)
    # 
    # We have implemented optimized vectorized version of the Kernels here so
    # that the svm training will run faster.
    print('Obliczam macierz jądra')
    if kernelFunction =='linearKernel':
        # to jądro można policzyć od razu dla wszystkich przykładów
        K = np.dot(X,X.T)
    else:
        # Jak nie możemy wymyśleć wektoryzacji obliczeń to
        # obliczamy każdy element macierzy jądra osobno
        K = np.zeros((m,m))
        for i in range(m):
            for j in range(i,m):
                K[i,j] = gaussianKernel(X[i,:].T, X[j,:].T,sigma)
                K[j,i] = K[i,j] #the matrix is symmetric
       
       
    print('Trenuję ...')
    dots = 12
    while passes < max_passes:        
        num_changed_alphas = 0
        for i in range(m): #dla każdego przykładu z ciągu uczącego
            # obliczamy błąd predykcji dla wektora i
            E[i] = b + np.sum (alphas*Y*K[:,i]) - Y[i]
            # jeśli jest co poprawiać:
            if (( (Y[i]*E[i] < -tol) & (alphas[i] < C)) | ((Y[i]*E[i] > tol) & (alphas[i] > 0))):
            
                # In practice, there are many heuristics one can use to select
                # the i and j. In this simplified code, we select them randomly.
                j = int(np.floor(m * np.random.rand()))
                while j == i:  # Make sure i \neq j
                    j = int(np.floor(m * np.random.rand()))
            
                # Obliczamy błąd predykcji dla wektora j.
                E[j] = b + np.sum (alphas*Y*K[:,j]) - Y[j]

                # Save old alphas
                alpha_i_old = alphas[i]
                alpha_j_old = alphas[j]
            
                # Oblicz przycięcia do pudełka [0,C] 
                if (Y[i] == Y[j]):
                    L = np.max((0, alphas[j] + alphas[i] - C))
                    H = np.min((C, alphas[j] + alphas[i]))
                else:
                    L = np.max((0, alphas[j] - alphas[i]))
                    H = np.min((C, C + alphas[j] - alphas[i]))
                if (L ==    H):
                    # continue to next i. 
                    continue
                # Compute eta by (15).
                eta = 2 * K[i,j] - K[i,i] - K[j,j]
                if (eta >= 0):
                    # continue to next i. 
                    continue
                # Compute and clip new value for alpha j using (16) and (17).
                alphas[j] = alphas[j] - (Y[j] * (E[i] - E[j])) / eta
            
                # Clip 
                alphas[j] = np.min ((H, alphas[j]))
                alphas[j] = np.max ((L, alphas[j]))
            
                #Check if change in alpha is significant
                if (np.abs(alphas[j] - alpha_j_old) < tol):
                    # continue to next i. 
                    # replace anyway
                    alphas[j] = alpha_j_old
                    continue
            
                # Determine value for alpha i using (16). 
                alphas[i] = alphas[i] + Y[i]*Y[j]*(alpha_j_old - alphas[j])
            
                # Compute b1 and b2 using (20) and (21) respectively. 
                b1 = b - E[i] - Y[i] * (alphas[i] - alpha_i_old) *  K[i,j] - Y[j] * (alphas[j] - alpha_j_old) *  K[i,j].T
                b2 = b - E[j] - Y[i] * (alphas[i] - alpha_i_old) *  K[i,j] - Y[j] * (alphas[j] - alpha_j_old) *  K[j,j].T

                # Compute b by (19). 
                if ( (0 < alphas[i]) & (alphas[i] < C)):
                    b = b1
                elif (0 < alphas[j]) & (alphas[j] < C):
                    b = b2
                else:
                    b = (b1+b2)/2
                num_changed_alphas = num_changed_alphas + 1    
        if (num_changed_alphas == 0):
            passes = passes + 1
        else:
            passes = 0
        print(num_changed_alphas)
    print(' Gotowe! \n\n')

    # Save the model
    idx = alphas > 0    
    model = {}
    model['X'] = X[idx,:]
    model['Y'] = Y[idx]
    model['kernelFunction'] = kernelFunction
    model['b'] = b
    model['alphas']= alphas[idx]
    model['w'] = (np.dot((alphas*Y).T, X)).T
    model['sigma'] = sigma
    print('ilość wektorów wspierających: ', len(model['alphas']))
    return model

#### Funkcja do wykonywania predykcji modelu wyuczonego w oparciu o poprzednią funkcję trenującą:

In [ ]:
def svmPredict(model,X):
    '''model - model otrzymany z funkcji svmTrain
     X - macierz m x n ,
                       w której wierszach są przykłady do sklasyfikowania (m)
                       każdy przykład ma wymiar n
     funkcja zwraca wektor pred - i-ty element to predykcja dla i-tego przykładu
    '''
    
    # pobieramy rozmiary:
    m,n = X.shape
    #print 'm,n',m,n
    # przygotowujemy tablice:
    pred = np.zeros(m) # predyktory
    margines = np.zeros(m)     # wartości marginesów
    if model['kernelFunction'] == 'linearKernel':
        margines = np.dot(X , model['w']) + model['b']
    elif model['kernelFunction'] =='gaussianKernel':
        for i in range(m): #ta pętla iteruje po przykładach z macierzy X
            for j in range(len(model['alphas'])): # ta pętlla iteruje po wektorach wspierających
                margines[i] += model['alphas'][j]*model['Y'][j]* gaussianKernel(X[i,:],model['X'][j,:],model['sigma'])
            margines[i] += model['b']
    else:
        print('niezaimplementowane jądro '+ model['kernelFunction'])
    
    pred[margines >= 0] =  1    
    pred[margines <  0] = -1
    
    return pred

Potrzebne będą nam też następujące zestawy danych:
* [Dane1.txt](https://brain.fuw.edu.pl/edu/images/c/ce/Dane1.txt)
* [Dane2.txt](https://brain.fuw.edu.pl/edu/images/3/3f/Dane2.txt)
* [Dane3.txt](https://brain.fuw.edu.pl/edu/images/3/33/Dane3.txt).

Proszę pobrać te pliki i zapisać je w bieżącym katalogu.

## Ćwiczenie 1: Dane separowalne liniowo
Poniższy kod prezentuje zastosowanie SVM do problemu, który jest separowalny liniowo.
Wykonując poniższy kod proszę zwrócić uwagę na punkt należący do klasy1 o współrzędnych (0.09, 4).


#### Wczytywanie danych

In [ ]:
dane = np.loadtxt('Dane1.txt') # dane zorganizowane są w trzech kolumnach
N_przyk, N_wej = dane.shape 
X = dane[:,0:2] # pierwsze dwie kolumny to wejście
y = dane[:,2] # trzecia kolumna to etykiety klas

#### Narysujmy te dane

In [ ]:
rysujDaneGrup(X, y, marker=('or','xb'), xlabel='x0', ylabel='x1',legend_list=('klasa0','klasa1'))
plt.show()

#### Trenujemy model

In [ ]:
model  = svmTrain(X, y, C=100, kernelFunction = 'linearKernel', tol = 1e-3, max_passes = 20,sigma = 10) 

#### Prezentujemy podział przestrzeni wejść reprezentowany przez model

In [ ]:
rysujDaneGrup(X, y, marker=('or','xb'), xlabel='x0', ylabel='x1',legend_list=('klasa0','klasa1'))
rysujPodzial(model,X)
plt.show()

#### A jak jest dla innych wartości C?
Jak pamiętamy z wykładu parametr C to współczynnik regularyzacji SVM, który karze za naruszanie marginesów. 
* Proszę wykonać  kod dla C o wartościach {1,2,5,10,20,30,60,120}  i zaobserwować wyniki.

## Ćwiczenie 2: jądro Gaussowskie
W  programie proszę zmodyfikować wywołanie <tt>svmTrain</tt>:
*  podmienić funkcję jądra na <tt>'gaussianKernel'</tt>.
* ustawić C = 10
* zmieniać sigma na wartości: {0.1, 0.2, 0.4, 0.8, 1, 2, 4, 8}

In [ ]:
model  = svmTrain(X, y, C=..., kernelFunction = ... , tol = 1e-3, max_passes = 20,sigma = ...) 
rysujDaneGrup(X, y, marker=('or','xb'), xlabel='x0', ylabel='x1',legend_list=('klasa0','klasa1'))
rysujPodzial(model,X)
plt.show()

## Ćwiczenie 3: skomplikowany podział nieliniowy

Przy pomocy kodu z ćwiczenia 2 proszę dobrać parametry C i sigma aby otrzymać sensownie wyglądający podział przestrzeni dla danych zawartych w pliku Dane2.txt.

In [ ]:
...

## Ćwiczenie 4: automatyzacja dobierania parametrów C i sigma 

W wielu prawdziwych zastosowaniach chcielibyśmy aby nasz wybór parametrów był optymalny a jednocześnie możliwie  obiektywny. 

Powszechnie stosowaną metodą jest przeszukanie przestrzeni parametrów (C,sigma). Generuje się siatkę wartości (C,sigma) i dla każdego punktu siatki:

* estymuje się model
* ocenia się jakość generalizacji 

Do oceny jakości w każdym punkcie siatki można zastosować albo  zbiór monitorujący albo metody typu leave-one-out.

**Uwaga**: podział przestrzeni często wykonuje się w skali logarytmicznej.

Ćwiczenie wykonamy dla zbioru uczącego z pliku dane3.txt.

* wczytywanie danych

In [ ]:
dane = np.loadtxt('dane3.txt') # dane zorganizowane są w trzech kolumnach
N_przyk, N_wej = dane.shape 
X = dane[:,0:2] # pierwsze dwie kolumny to wejście
y = dane[:,2] # trzecia kolumna to etykiety klas

Musimy ten zbiór podzielić na dane do trenowania i dane do testowania np. w proporcji 3:1, Można to zrobić tak:

* Rozdzielamy dane na grupy:

In [ ]:
grupa0, = np.where(y==-1)
grupa1, = np.where(y==1)

* mieszamy kolejność indexów w każdej grupie

In [ ]:
np.random.shuffle(grupa0)
np.random.shuffle(grupa1)

* kopiujemy dane do zbioru uczącego (pierwsze 75% grupy0 i grupy1)

In [ ]:
Xu = X[np.concatenate((grupa0[0: int(0.75*len(grupa0))],grupa1[0:int(0.75*len(grupa0))]))]
yu = y[np.concatenate((grupa0[0: int(0.75*len(grupa0))],grupa1[0:int(0.75*len(grupa0))]))]

* kopiujemy dane do zbioru testowego (końcowe 25% grupy0 i grupy1)

In [ ]:
Xt = X[np.concatenate((grupa0[int(0.75*len(grupa0)):], grupa1[int(0.75*len(grupa0)):]))]
yt = y[np.concatenate((grupa0[int(0.75*len(grupa0)):], grupa1[int(0.75*len(grupa0)):]))]

* narysujmy te dane

In [ ]:
rysujDaneGrup(Xu, yu, marker=('xr','xb'), xlabel='x0', ylabel='x1',legend_list=('klasa0','klasa1'))
rysujDaneGrup(Xt, yt, marker=('or','ob'), xlabel='x0', ylabel='x1',legend_list=('klasa0_test','klasa1_test'))
plt.show()

Mając taki podział danych możemy dopasować model SVM do części uczącej:

In [ ]:
model  = svmTrain(Xu, yu, C=10, kernelFunction = 'gaussianKernel', tol = 1e-3, max_passes = 20,sigma = 0.5) 

A następnie ocenić jego jakość na części testowej (funkcja <tt>svmPredict</tt> dostarczana jest przez moduł svm_modul.py):

In [ ]:
TPR = np.sum(yt == svmPredict(model,Xt))/float(len(yt))
print(TPR)

#### Proszę napisać kod, który
* skanuje przestrzeń (C,sigma): C w zakresie od 0.1 do 100, sigma w zakresie od 0.1 do 10. Do wygenerowania zakresu ze skalą logarytmiczną można wykorzystać np. takie polecenie: <tt>zakresC = np.logspace(np.log2(0.1),np.log2(100),8, base=2)</tt>
* znajduje najlepsze parametry
* rysuje podział przestrzeni dla najlepszych parametrów.